# [Obspy](https://docs.obspy.org/): a Python Toolbox for seismology

## ROSES Tutorial Outline

1. Reading data from a file
2. Downloading data from online repositories
3. Removing instrument response
4. Writing data to a file
5. Some Obspy stream and trace methods
6. Plotting with Matplotlib

### Instructor: Sydney Dybing (sdybing@uoregon.edu)

#### Dependencies needed: Obspy, Numpy, Matplotlib

Since this lecture is only an hour, I intend for it to serve as a basic introduction to using Obspy. I've written it to include information about how to use the tools I find myself using most often in my research.

If you want to learn more or there are functions I didn't cover that you need, there are several fantastic options for learning more about Obspy online. In particular, I will link the official [Obspy Tutorial](https://docs.obspy.org/tutorial/index.html), and I also recommend the [Seismo-Live Juypter Notebooks for Seismology](https://krischer.github.io/seismo_live_build/tree/index.html), which includes an Obspy section among other topics. 

I'll start showing you some of the basic capabilities of Obspy with a sample data file that I already downloaded, and was provided on Slack. This is in miniSEED format, but Obspy can also automatically detect many other data formats, such as SAC.

## Reading data from a file

With the read function, you basically just only need the path to the file on your computer. There are additional options, which you can check out at the Obspy documentation page for the [read function](https://docs.obspy.org/packages/autogen/obspy.core.stream.read.html).

The data is read into a [stream object](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.html#obspy.core.stream.Stream).

In [ ]:

print()

Stream objects are basically collections of trace objects, which contain the data and associated metadata. You can grab a trace from a stream element the way you would an item from a list.

In [ ]:

print()

To view the data:

In [ ]:

print()

This is a numpy array, so you can do any operations then with this array that you normally would in Python.

To view the metadata:

In [ ]:
print()

If you want to use any of these particular things, you can grab them individually as follows:

In [ ]:
print()
print()

You can also make simple plots of traces.

You can change these simple Obspy plots to look more unique and plot different things. More info can be found in the [Obspy tutorial pages online](https://docs.obspy.org/tutorial/code_snippets/waveform_plotting_tutorial.html).

Here is an example of changing the color:

So far, we've been working with just one trace from a stream, which has a single channel of data from a single station. Obspy streams are useful because they can hold multiple traces. You can read them in several different ways.

First, you can add individual files to a stream one at a time.

In [ ]:

print()

Another option is to read individual files in as traces, and then add them to a stream. The output is ultimately the same.

In [ ]:

print()

You can make simple plots of stream objects the same way you can with traces.

To convert our y-axis units from digital counts to actual ground motion, we need to remove the instrument response, which I'll discuss more later. You can download responses alongside data. So let's talk about downloading data from online repositories like the IRIS Data Management Center.

## Downloading data from online repositories

IRIS is just one "client" that stores data, and other client options can be found [here](https://docs.obspy.org/packages/obspy.clients.fdsn.html). [FDSN](https://www.fdsn.org/about/) is the International Federation of Digital Seismograph Networks, which helps coordinate global data access.

Times are managed in Obspy using a class called UTCDateTime. 

UTCDateTime objects are easy to use and the format is easy to understand. If you're like me and you ocassionally forget exactly how they're formatted, it's simple to check the Obspy documentation [here](https://docs.obspy.org/tutorial/code_snippets/utc_date_time.html).

To create a UTCDateTime object, you just need a date and a time, and to string them together in the right order, which is as follows:

"YYYY-MM-DDTHH:MM:SS"

If you're working in juldays, which is just counting the day of the year out of 365, with January 1 being day 001, this is the format:

"YYYY-DDDTHH:MM:SS"

Let's try an example from last year's [M7.1 Ridgecrest earthquake](https://earthquake.usgs.gov/earthquakes/eventpage/ci38457511/executive).

In [ ]:

print()

You can add or subtract a number of seconds from a UTCDateTime object, which is useful for downloading a specific segment of data.

In [ ]:

print()

In [ ]:

print()

Great! We have some times - so where do we want to get the data from?

The widely-adopted [SEED standard](https://ds.iris.edu/ds/nodes/dmc/data/formats/seed/) for data follows this convention for data identification:

__Network code__: Identifies which network the data belongs to. You can check out all of the available networks at IRIS MetaData Aggregator [here](http://ds.iris.edu/mda/). Network codes are unique and assigned by the FDSN.

__Station code__: The station within a network - these aren't necessarily unique, so you need to use network codes with station codes to identify a station.

__Location ID__: Sometimes stations can have more than one instrument at them, which is specified by the location ID.

__Channel codes__: Three character code - Emily will talk more about this next week!

Let's start with a simple example, where we choose one network, one station, one location (one instrument basically that's part of the station), and one channel. Here is the [MDA page](http://ds.iris.edu/mda/IU/TUC/?starttime=1992-06-13&endtime=2599-12-31) for the Tucson, AZ station we will use. 

To actually download the data, we can use an Obspy function called get_waveforms.

In [ ]:

print()


If we want to download multiple channels, in Obspy you can use Linux syntax like wildcards to accomplish this.

In [ ]:

print()


## Removing instrument response

This data right now is quantitatively meaningless because we haven't removed the instrument response. 

Theoretically, this is relatively complicated (see [Havskov and Alguacil, 2015](https://books.google.com/books?id=5PPuCgAAQBAJ&pg=PA197&lpg=PA197&dq=10.1007/978-3-319-21314-9_6&source=bl&ots=R_XJrZxu59&sig=ACfU3U2YdUF5_nlVwRFs0Hbvdm5fHI7_Xw&hl=en&sa=X&ved=2ahUKEwjc7sjft5bqAhWSJzQIHYS7A0UQ6AEwCnoECAoQAQ#v=onepage&q=10.1007%2F978-3-319-21314-9_6&f=false)). In practice, Obspy makes correcting for instrument response easier! You just have to make sure you download the response with the data.

We'll keep working with the same station and earthquake. As a refresher, here are the variables we have set. We'll stick with working on one channel for now to make visualization simpler.

In [ ]:
time = UTCDateTime("2019-07-06T03:19:53.04")
starttime = time - 60 
endtime = time + 60*15

net = "IU"
sta = "TUC"
loc = "00"
chan = "HH1"

Now, all we have to do is add one option when we use get_waveforms:

In [ ]:
st = client.get_waveforms(net, sta, loc, chan, starttime, endtime)
print(st)
st.plot();

It doesn't look any different yet, but the responses have been downloaded. Before removing the response, you might want to copy your data, since the function remove_response acts directly on the data, and will overwrite the original. Copying a stream is simple:

In [ ]:

print()
print()

Now we can remove the response. You can set which units you want the ground motion to be represented in using the output option.

In [ ]:


# Remember, if you run this cell multiple times, your output will be strange because you already removed the
# response from st_rem. If you want to do it again and try something else, you either have to make a new copy
# of the original st again, or go back and re-run the previous cell that copied st.

This is an oversimplification of the response removal process.

We can visualize what remove_response is doing by using the plot = True option. This is what it looks like if we don't do any kind of filtering.

In [ ]:
# repeating this since the last cell will have removed the response from st_rem already

# other options: output = 'DISP', 'ACC'

This is what our data looks like now:

## Writing downloaded data to a file

This can be done with st.write() - you just specify the desired file path, name, extension, and data format (SAC, MSEED, etc.). It looks something like this:

stream.write('/path/filename.mseed', format='MSEED')

For this example:

Now we can read it in as a stream just as we did with the sample file at the beginning of this tutorial.

In [ ]:

print()


## Some Stream and Trace Methods

The stream and trace objects in Obspy both have a number of public methods that can be used to modify the data. I'll go over a few of them that I use most often, and if there is anything else you're interested in doing, you can check the Obspy documentation for [traces](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.html#obspy.core.trace.Trace) and for [streams](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.html).

### Filtering

One thing we might want to do is look at a specific frequency range of data in our earthquake. We can use the "filter" method for this. There are many different filter methods available, and all of these options can be found in the Obspy documentation [here](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.filter.html#obspy.core.stream.Stream.filter).

Let's try a bandpass example:

### Trimming data

You can shorten a stream and remove unwanted data with the ["trim"](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.trim.html#obspy.core.stream.Stream.trim) method. For this method, you need UTCDateTime objects for the time you want your new trimmed trace to start and end. 

Let's refresh our memories about the times in this data:

In [ ]:
print()

So let's say we want to chop 6 minutes off the end of this trace, and 2 minutes off the front.

Don't forget to copy the stream again if you want to keep the original.

While trimming data, you also have an option to fill gaps with a value, and for this you just set fill_value=somenumber in the method after defining the start and end time.

### Changing sampling rates

There are three methods in Obspy for changing the sampling rates of data in a stream or trace:

1. [Decimate](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.decimate.html#obspy.core.stream.Stream.decimate): downsamples data by an integer factor
2. [Interpolate](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.interpolate.html#obspy.core.trace.Trace.interpolate): increase sampling rate by interpolating (many method options)
3. [Resample](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.resample.html#obspy.core.stream.Stream.resample): resamples data using a Fourier method

I'll demonstrate using decimate. Let's refresh our memories about this data's sampling rate:

Let's decimate by a factor of 10.

If we check the stats again, you can see the sampling rate is different!

Other methods you might be interested in using: differentiate, integrate, stack, merge, sort, and many others. To see how to use these, check the Obspy documentation for streams and traces, which I linked at the beginning of this section, as there are lists and links to how to implement each of them!

## Matplotlib

Let's examine how we can make some basic plots of seismic data in [Matplotlib](https://matplotlib.org/).

We need to separate from the stream the times (independent variable) and the data (dependent variable).

In [ ]:


print()

Just as before, this pulls the data out into a Numpy array that can then be used for plotting.

Getting the times is slightly different in that it's a method (["times"](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.times.html#obspy.core.trace.Trace.times)) instead of a property.

In [ ]:


print()

Because these both come from the same data trace, the arrays should be the same length, and we should be able to stick them in a Matplotlib plot easily.

In [ ]:
print()
print()

Great! Let's try it out:

Here is what I might do to clean up this basic plot a bit:

If you want to plot the data from three channels from the same instrument at the same time, you can use subplots. 

To avoid confusion, let's just go back and re-download the data from this earthquake, and include all three channels. This code is copied from earlier in the notebook - the only difference is that I'll remove the response now from all three channels.

In [ ]:
time = UTCDateTime("2019-07-06T03:19:53.04")
starttime = time - 60 
endtime = time + 60*15

net = "IU"
sta = "TUC"
loc = "00"
chan = "HH*"

st = client.get_waveforms(net, sta, loc, chan, starttime, endtime, attach_response = True)
print(st)

st.remove_response(output = 'VEL')

st.plot();

Alright! Let's plot each of these three channels on their own subplots. First, we have to separate each trace's data and times out.

We can then define them as such.

Now we can start plotting. We'll start by making a big "main" figure, and then we add subplots and the data to each of them.

To define subplots in Matplotlib, for each iteration you list the number of rows and number of columns you want, as well as which one you're on.

Let's try making the figure bigger. We can define a size in inches, with the width first and the height second.

In [ ]:
fig = plt.figure()

plt.subplot(311)
plt.plot(HH1_times, HH1_data)

plt.subplot(312)
plt.plot(HH2_times, HH2_data)

plt.subplot(313)
plt.plot(HHZ_times, HHZ_data);

Now let's clean it up with some labels, like we did last time, and add some other features like legends and colors.

In [ ]:
fig = plt.figure(figsize = (8,10))


You could make this plot even better with more advanced Matplotlib capabilities that you can look up and mess with. 

Here is one final useful Matplotlib capability, with code provided by Liam Toney (who will be teaching the PyGMT tutorial): plotting so that you have your UTC times on the x-axis, rather than just seconds from zero, using [matplotlib.dates](https://matplotlib.org/api/dates_api.html).

We'll just demonstrate this on one channel - HH1. When we calculate the times this time, instead of leaving the parentheses blank, we specify the "type" of times as "matplotlib" so that we can plot them in UTC time.

Much of the plotting here is the same - it just incorporates another Matplotlib capability where you build your axes more manually. You can check out the documentation for this [here](https://matplotlib.org/api/axes_api.html). The basic plotting setup is as follows:

Now we need to make the dates on the x-axis actually useful. We can do this with a locator. You can read more about this at the matplotlib.dates documentation linked above, but what this code does is allow Matplotlib to pick the best locations for the ticks on the x-axis and the best format for them. 

In [ ]:
fig, ax = plt.subplots()

ax.plot(HH1_times_mpl, HH1_data)
ax.set_xlim(HH1_times_mpl[0], HH1_times_mpl[-1])
ax.set_xlabel("UTC Time")
ax.set_ylabel("Ground Velocity (m/s)")
ax.set_title("M7.1 Ridgecrest Earthquake - " + net + "." + sta + "." + loc + "." + chan);



There we go!

I hope this notebook serves as a good introduction and starting point to working with seismic data in Obspy, and a resource for your future coding needs!